In [1]:
# Initialization
x = [-np.pi/4, np.pi/2, np.pi/4] # Initial robot state

# Control loop definition
def control(x, param):
    f = fkin(x, param)  # param:parameter 参数
    J = Jkin(x, param)

    print('\n f:',f)
    print('J:',J)
    u = np.linalg.pinv(J) @ logmap(param.Mu, f) # Position & orientation tracking
    print('u:', u)
    u1 = np.linalg.pinv(J[:2,:]) @ (param.Mu[:2] - f[:2]) # Position tracking
    print('u1:', u1)
    u2 = np.linalg.pinv(J[2:,:]) @ (param.Mu[2:] - f[2:]) # Orientation tracking
    print('u2:', u2)
    # u = J.T @ logmap(param.Mu, f) * 1E-4 # Gradient-based tracking
    # u = np.zeros(param.nbVarX) # Zero control commands

    return 0.1 * u / param.dt   # Velocity in rad/s

NameError: name 'np' is not defined

In [2]:
# Initialization
x = [-np.pi/4, np.pi/2, np.pi/4] # Initial robot state

# Control loop definition
def control(x, param):
    f = fkin(x, param)
    J = Jkin(x, param)

    print('\n f:',f)
    print('J:',J)
    pinvJ = np.linalg.inv(J.T @ J + np.eye(param.nbVarX) * 1E1) @ J.T # Damped pseudoinverse 阻尼伪逆
    # array([[10.,  0.,  0.],
    #       [ 0., 10.,  0.],
    #       [ 0.,  0., 10.]])
    print(np.eye(param.nbVarX)*1E1)
    u = pinvJ @ logmap(param.Mu, f) # Position & orientation tracking
    print(u)

    return 0.1 * u / param.dt   # Velocity in rad/s

NameError: name 'np' is not defined

In [3]:
1E-2

0.01

In [4]:
# Initialization
x = [-np.pi/4, np.pi/2, np.pi/4] # Initial robot state

# Control loop definition
def control(x, param):
    f = fkin(x, param)
    J = Jkin(x, param)

    # Weights in task space
    Wf = np.diag([1, 1, 0])  # 优先跟踪第一，第二关节
    pinvWJ = np.linalg.inv(J.T @ Wf @ J + np.eye(param.nbVarX) * 1E-2) @ J.T @ Wf # Weighted pseudoinverse  # 只迭代position
    u = pinvWJ @ logmap(param.Mu, f) # Position & orientation tracking

    # # Weights in configuration space
    # Wx = np.diag([0.01, 1, 1])
    # pinvWJ =  Wx @ J[:2,:].T @ np.linalg.inv(J[:2,:] @ Wx @ J[:2,:].T + np.eye(2) * 1E-2) # Weighted pseudoinverse
    # u = pinvWJ @ (param.Mu[:2] - f[:2]) # Position tracking

    return 0.1 * u / param.dt   # Velocity in rad/s

NameError: name 'np' is not defined

In [ ]:
# Initialization
x = np.array([-np.pi/4, np.pi/2, np.pi/4]) # Initial robot pose
xh = np.array([-np.pi/4, np.pi/2, np.pi/4]) # Preferred robot pose

# Control loop definition
def control(x, param):
    f = fkin(x, param)
    J = Jkin(x, param)

    # Prioritized tracking with motion of first joint as secondary task # 优先跟踪，第一关节运动作为次要任务
    u1 = np.linalg.pinv(J[:2,:]) @ (param.Mu[:2] - f[:2]) # Position tracking
    N = np.eye(param.nbVarX) - np.linalg.pinv(J[:2,:]) @ J[:2,:] # Nullspace projection matrix
    u = u1 + N @ [1, 0, 0] # Control commands

    # # Prioritized tracking with preferred pose maintenance as secondary task # 优先跟踪，首选姿势维护为次要任务
    # u1 = np.linalg.pinv(J[:2,:]) @ (param.Mu[:2] - f[:2]) # Position tracking
    # N = np.eye(param.nbVarX) - np.linalg.pinv(J[:2,:]) @ J[:2,:] # Nullspace projection matrix
    # u = u1 + N @ (xh - x) # Control commands

    # # Prioritized control (position tracking prioritized over orientation tracking) # 优先控制（位置跟踪优先于方向跟踪）
    # dfp = (param.Mu[:2] - f[:2]) * 10 # Position correction
    # dfo = (param.Mu[2:] - f[2:]) * 10 # Orientation correction
    # Jp = J[:2,:] # Jacobian for position
    # Jo = J[2:,:] # Jacobian for orientation
    # pinvJp = np.linalg.inv(Jp.T @ Jp + np.eye(param.nbVarX) * 1e-2) @ Jp.T # Damped pseudoinverse
    # Np = np.eye(param.nbVarX) - pinvJp @ Jp # Nullspace projection operator
    # up = pinvJp @ dfp # Command for position tracking
    # JoNp = Jo @ Np
    # pinvJoNp = JoNp.T @ np.linalg.inv(JoNp @ JoNp.T + np.eye(1) * 1e1) # Damped pseudoinverse
    # uo = pinvJoNp @ (dfo - Jo @ up) # Command for orientation tracking (with position tracking prioritized) # 定向跟踪命令（优先考虑位置跟踪）
    # u = up + Np @ uo # Control commands

    return 0.1 * u / param.dt   # Velocity in rad/s